In [1]:
!curl http://localhost:9200/

{
  "name" : "node-1",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "Hd5owN6ERfSiT3DlTZTENg",
  "version" : {
    "number" : "8.2.2",
    "build_flavor" : "default",
    "build_type" : "deb",
    "build_hash" : "9876968ef3c745186b94fdabd4483e01499224ef",
    "build_date" : "2022-05-25T15:47:06.259735307Z",
    "build_snapshot" : false,
    "lucene_version" : "9.1.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [2]:
import elasticsearch 
import json

from elasticsearch import helpers
from collections import deque

In [3]:
def data_gen(): 
    with open('./data_hub/data.json', 'r') as f:
        data = json.load(f)  
           
    for i in data:
        json_data = {}
        json_data['title'] = i['title']
        json_data['text'] = i['text']
        json_data['url'] = i['url']
        yield json_data 

In [4]:
# Initiate Elasticsearch
es = elasticsearch.Elasticsearch('http://127.0.0.1:9200/')

# Bulk load data into Elasticsearch
es.indices.create(index='dis_docs')
deque(helpers.parallel_bulk(es, data_gen(), index='dis_docs'), maxlen=0)
es.indices.refresh()

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [6]:
# Search documents in Elasticsearch
query = {'query':{'match_phrase':{'text':'Plague of Cyprian'}}}

hits = es.search(index='dis_docs', body=query) 
hits['hits']

/tmp/ipykernel_21077/2386824436.py:4: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  hits = es.search(index='dis_docs', body=query)


{'total': {'value': 1, 'relation': 'eq'},
 'max_score': 6.7403617,
 'hits': [{'_index': 'dis_docs',
   '_id': 'nIngU4EB5DPWiFiNgAUE',
   '_score': 6.7403617,
   '_ignored': ['text.keyword'],
   '_source': {'title': 'Plague of Cyprian',
    'text': 'The Plague of Cyprian was a pandemic that afflicted the Roman Empire about from AD 249 to 262. The plague is thought to have caused widespread manpower shortages for food production and the Roman army, severely weakening the empire during the Crisis of the Third Century. Its modern name commemorates St. Cyprian, bishop of Carthage, an early Christian writer who witnessed and described the plague. The agent of the plague is highly speculative because of sparse sourcing, but suspects have included smallpox, pandemic influenza and viral hemorrhagic fever (filoviruses) like the Ebola virus.',
    'url': 'https://en.wikipedia.org/wiki/Plague_of_Cyprian'}}]}